In [1]:
import cv2

In [2]:
cv2.TrackerBoosting_create()

<TrackerBoosting 00000112E1ECC570>

In [35]:
import cv2
import numpy as np
import math
import typing
import os
def show(frame):
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def plot_point(frame,x,y,color=(0,255,0)):
    radius = 2
    thickness = -1
    return cv2.circle(frame, (x,y), radius, color, thickness)
def plot_line(frame,p1,p2,color=(0,191,255)):
    thickness = 2
    return cv2.line(frame, (p1[0],p1[1]), (p2[0],p2[1]), color, thickness)
def getIdealPointGroup(points):
  pointGroups = []
  index = 0
  subgroup = [points[0]]

  for i in range(len(points) - 1):
    prevPoint = points[i]
    currentPoint = points[i+1]

    if (abs(int(prevPoint[0])-int(currentPoint[0])) <= 1) and (abs(int(prevPoint[1])-int(currentPoint[1])) <= 1):
      subgroup.append(currentPoint)
    else:
      pointGroups.append(subgroup[:])
      subgroup = [currentPoint]

  pointGroups.append(subgroup)

  mainPointGroup = []
  maxPointGroupSize = 0

  for group in pointGroups:
    if len(group) > maxPointGroupSize:
      maxPointGroup = group
      maxPointGroupSize = len(group)

  return maxPointGroup

# Gets all the contours for certain image
  

def get_bottom_left(points,img,index = None):
  if not index is None:
    i = int(index*len(points))
    return points[i], index 
  max_y = len(img[0])
  index = 0
  dist = 100000
  count = 0
  for i in points:
    if (i[0])**2+(max_y-i[1])**2<dist:
      index = count
      dist = (i[0])**2+(max_y-i[1])**2
    count+=1
  return points[index],index/len(points)

def loadvideo(filename: str) -> np.ndarray:
    """Loads a video from a file.

    Args:
        filename (str): filename of video

    Returns:
        A np.ndarray with dimensions (channels=3, frames, height, width). The
        values will be uint8's ranging from 0 to 255.

    Raises:
        FileNotFoundError: Could not find `filename`
        ValueError: An error occurred while reading the video
    """

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_width, frame_height, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        v[count] = frame

    v = v.transpose((3, 0, 1, 2))

    return v
def savevideo(filename: str, array: np.ndarray, fps: typing.Union[float, int] = 1):
    """Saves a video to a file.

    Args:
        filename (str): filename of video
        array (np.ndarray): video of uint8's with shape (channels=3, frames, height, width)
        fps (float or int): frames per second

    Returns:
        None
    """

    c, f, height, width = array.shape

    if c != 3:
        raise ValueError("savevideo expects array of shape (channels=3, frames, height, width), got shape ({})".format(", ".join(map(str, array.shape))))
    fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
    out = cv2.VideoWriter(filename, fourcc, fps, (width, height))

    for i in range(f):
        out.write(array[:, i, :, :].transpose((1, 2, 0)))

        
def obtainContourPoints(img):
  # read image

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # set lower and upper bounds on blue color
  lower = (0,0,250)
  upper = (200,200,255)

  # threshold and invert so hexagon is white on black background
  thresh = cv2.inRange(rgb, lower, upper)
  #show(thresh)
  # get contours
  result = np.zeros_like(thresh)
  im2, contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  #print(im2,contours)
  #cv2.drawContours(img, im2, -1, (0,255,0), 3)

  im2 = np.array(im2)
  index_1 = 0
  index_2 = 0
  for i in im2:
    if i.shape[0]>im2[index_1].shape[0]:
      index_1=index_2
    index_2+=1
  rec = cv2.minAreaRect(im2[index_1])
  color = (0, 0, 255) 
  thickness = 2
  box = cv2.boxPoints(rec)
  box = np.int0(box)
  indexes = [0,1]
  for i in range(0,len(box)):
    for k in range(0,len(indexes)):
      if box[i][1]>box[indexes[k]][1] and not i in indexes:
        indexes[k]=i

  #cv2.drawContours(img,[box],0,(191,0,255),2)
  img = plot_line(img,box[indexes[0]],box[indexes[1]])
  img = plot_point(img,box[indexes[0],0],box[indexes[0],1])
  img = plot_point(img,box[indexes[1],0],box[indexes[1],1])
  #cv2.rectangle(img,rec,color,thickness)
  return img, [box[indexes[0]].tolist(),box[indexes[1]].tolist()]

In [46]:
key_points = []

In [47]:
from tqdm import tqdm
folder = 'C:\\Users\\TheurerJ\\Documents\\Data\\Good_Vids'
output = 'C:\\Users\\TheurerJ\\Documents\\Data\\vids with lines'
for vid in tqdm(os.listdir(folder)):
    video_file = os.path.join(folder,vid)
    first = loadvideo(video_file)
    first = np.transpose(first,(1,2,3,0))
    video = []
    guess = None
    vertexes = []
    for frame in first:
        img,points = obtainContourPoints(frame)
        vertexes.append(points)
        video.append(img)
    video = np.transpose(np.array(video),(3,0,1,2))
    key_points.append(np.array(vertexes))
    savevideo(os.path.join(output,vid),video,fps=30)


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 16.51it/s]


In [50]:
key_points[0][:,0,:]

array([[77, 71],
       [62, 69],
       [80, 68],
       [62, 69],
       [60, 71],
       [79, 72],
       [60, 71],
       [58, 73],
       [60, 73],
       [80, 73],
       [80, 72],
       [58, 73],
       [60, 71],
       [60, 71],
       [58, 71],
       [60, 71],
       [60, 71],
       [78, 72],
       [60, 70],
       [62, 69],
       [62, 69],
       [62, 71],
       [60, 71],
       [58, 71],
       [60, 73],
       [60, 73],
       [62, 72],
       [58, 71],
       [58, 73],
       [58, 71],
       [60, 71],
       [60, 71],
       [60, 71],
       [60, 69],
       [60, 69],
       [60, 69],
       [62, 69],
       [62, 69],
       [62, 71],
       [80, 70],
       [60, 71],
       [60, 71],
       [80, 72],
       [62, 71],
       [62, 71],
       [60, 71],
       [60, 71],
       [60, 71],
       [60, 71],
       [60, 69],
       [60, 69],
       [60, 69],
       [60, 69],
       [60, 69],
       [62, 69],
       [62, 68],
       [80, 72],
       [60, 70],
       [78, 72

In [41]:
np.array(key_points).shape

(22,)